In [ ]:
!pip install openai==0.28 # openai 라이브러리를 설치합니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00


In [ ]:
import os
import openai
import json
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
import requests

def get_current_location():
    response = requests.get('https://ipinfo.io')
    info = response.json()
    return json.dumps(info)

In [ ]:
from datetime import datetime

def get_current_datetime():
    now = datetime.now()
    info = {
        "date": now.strftime("%Y-%m-%d"),
        "time" : now.strftime("%H:%M:%S")
    }
    return json.dumps(info)

In [ ]:
import requests

def get_current_weather(latitude, longitude):

    # Define the base URL for the Open-Meteo API
    base_url = "https://api.open-meteo.com/v1/forecast"

    # Specify your parameters (in this case, for Berlin, Germany)
    params = {
        "latitude": latitude,   # Example latitude
        "longitude": longitude,  # Example longitude
        "current_weather": True, # To get current weather
        "daily": "temperature_2m_max,temperature_2m_min", # For daily max/min temperatures
        "timezone": "auto" # Automatically determine the timezone
    }

    # Make the GET request
    response = requests.get(base_url, params=params)

    info = {}
    # Check if the request was successful
    if response.status_code == 200:
        info = response.json()

    return json.dumps(info)

In [ ]:
def get_event_schedule():

    info = {
        'event_importance' : 'formal',
        'activity_level' : 'sitting'
    }

    return json.dumps(info)

In [ ]:
def get_my_info():

    info = {
        'sex' : 'male',
        'age' : '44'
    }

    return json.dumps(info)

In [ ]:
def recommend_clothing(sex, age, temperature, weather_type, event_importance, activity_level):

    # Crafting a message for the system's content to describe what the function should do
    system_message = "Given the user's sex, current temperature, weather type, event importance, and activity level, recommend appropriate clothing options."

    # Packing the user's input into a single message
    user_message = f"User's sex: {sex}, age : {age}, Temperature: {temperature}, Weather type: {weather_type}, Event importance: {event_importance}, Activity level: {activity_level}"

    # Assuming 'chat_history' should actually be 'messages' based on the provided code snippet
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

    # Assuming 'functions' and 'function_call' are part of a larger implementation not shown here
    # As 'openai.ChatCompletion.create' is not a directly available method in the OpenAI API Python library,
    # this example is structured to illustrate how you might structure the data for such a call.
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo-0613",
                                                messages=messages)

    # Extracting the recommended clothing options from the response
    output = response["choices"][0]["message"]["content"]

    return output


In [ ]:
functions = [
    {
        "name": "get_current_datetime",
        "description": "Get the current datetime.",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_current_location",
        "description": "Get the current location like latitude and longitude.",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_current_weather",
        "description": "Get the current weather like temperature and weather type in a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "string",
                    "description": "latitude",
                },
                "longitude": {
                    "type": "string",
                    "description": "longitude",
                },
            },
            "required": ["latitude", "longitude"],
        }
    },
    {
        "name": "get_event_schedule",
        "description": "Get event importance and activity level",
        "parameters": {},
        "required": [],
    },
    {
        "name": "get_my_info",
        "description": "Get my information sex and age",
        "parameters": {},
        "required": [],
    },
    {
        "name": "recommend_clothing",
        "description": "It is used when recommending clothes.",
        "parameters": {
            "type": "object",
            "properties": {
                "sex": {
                    "type": "string",
                    "description": "sex by calling get_my_info()",
                },
                "age": {
                    "type": "string",
                    "description": "age by calling get_my_info()",
                },
                "temperature": {
                    "type": "string",
                    "description": "current temperature using callling get_current_weather function",
                },
                "weather_type": {
                    "type": "string",
                    "description": "current weather type. rainy, sunny, snowy, etc. using callling get_current_weather function",
                },
                "event_importance": {
                    "type": "string",
                    "description": "casual, formal, or sporty events based on their importance or nature. using calling get_event_schedule()",
                },
                "activity_level": {
                    "type": "string",
                    "description": "sitting in a conference vs. a hiking trip. using calling get_event_schedule()",
                },
            },
            "required": ["sex", "temperature", "weather_type", "event_importance", "activity_level"],
        }
    },
]

In [ ]:
toolkits = {
    "get_current_datetime": get_current_datetime,
    "get_current_location": get_current_location,
    "get_current_weather": get_current_weather,
    "get_event_schedule" : get_event_schedule,
    "get_my_info" : get_my_info,
    "recommend_clothing" : recommend_clothing,
}

In [ ]:
system_prompt = """
Act as helpful assistant. Write in Korean.
To handle user requests, we determine which function to call first and try to acquire the necessary information from the given list of functions as much as possible.
"""

def print_message(message):
    print(json.dumps(message, indent=4))
    #if message.get("function_call"):
    #    print(message['function_call']['name'])

def openai_llm(input_text, chat_history):

    if len(chat_history) == 0:
        chat_history.append({"role": "system", "content": system_prompt})

    # input_text : 안녕
    chat_history.append({"role": "user", "content": input_text})

    while True: # 함수 호출을 더 이상하지 않을 때까지 루프

        response = openai.ChatCompletion.create(model="gpt-3.5-turbo-0613",
                                                messages=chat_history,
                                                functions=functions,
                                                function_call="auto")

        response_message = response["choices"][0]["message"]

        # AI의 응답을 추가
        chat_history.append(response_message)

        print_message(response_message)

        # function_call의 조건
        # 1. 함수 호출을 하지 못할 때, 필요한 인자가 메시지의 리스트에 없을 경우 (편지봉투 내에) > 사용자에게 함수 인자에 대한 정보를 요청
        # 2. 함수 호출을 할 필요가 없을 때 > 함수 호출 결과를 요약
        if response_message.get("function_call") == None:
            break

        # 함수 이름과 인자 값을 얻어온다.
        function_name = response_message["function_call"]["name"]
        function_arguments = json.loads(response_message["function_call"]["arguments"])

        # 함수를 호출해서, 그 결과를 얻어온다.
        function_response = toolkits[function_name](**function_arguments)

        function_response_message = {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }

        # 함수 호출 결과를 추가한다.
        chat_history.append(function_response_message)
        print_message(function_response_message)

    output = response.choices[0].message.content # function_call 키가 있다면, content는 null이나, function_call 키가 없을 때, break되므로 항상 content가 있다.

    return output

In [ ]:
print(openai_llm("옷 추천해줘", []))

{
    "role": "assistant",
    "content": null,
    "function_call": {
        "name": "get_my_info",
        "arguments": "{}"
    }
}
{
    "role": "function",
    "name": "get_my_info",
    "content": "{\"sex\": \"male\", \"age\": \"44\"}"
}
{
    "role": "assistant",
    "content": null,
    "function_call": {
        "name": "get_current_location",
        "arguments": "{}"
    }
}
{
    "role": "function",
    "name": "get_current_location",
    "content": "{\"ip\": \"35.221.61.237\", \"hostname\": \"237.61.221.35.bc.googleusercontent.com\", \"city\": \"Washington\", \"region\": \"Washington, D.C.\", \"country\": \"US\", \"loc\": \"38.8951,-77.0364\", \"org\": \"AS396982 Google LLC\", \"postal\": \"20004\", \"timezone\": \"America/New_York\", \"readme\": \"https://ipinfo.io/missingauth\"}"
}
{
    "role": "assistant",
    "content": null,
    "function_call": {
        "name": "get_current_weather",
        "arguments": "{\n  \"latitude\": \"38.8951\",\n  \"longitude\": \"-77.036

In [ ]:
def chat_with_user(user_message, chat_history):
    ai_message = openai_llm(user_message, chat_history)
    return ai_message

chat_history = []

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message, chat_history)
    print(f" A I > {ai_message}")

USER > 안녕
{
    "role": "assistant",
    "content": "\uc548\ub155\ud558\uc138\uc694! \ub3c4\uc6c0\uc774 \ud544\uc694\ud558\uc2e0\uac00\uc694? \uc81c\uac00 \ub3c4\uc640\ub4dc\ub9b4 \uc218 \uc788\ub294 \ub0b4\uc6a9\uc774 \uc788\uc73c\uba74 \ub9d0\uc500\ud574\uc8fc\uc138\uc694."
}
 A I > 안녕하세요! 도움이 필요하신가요? 제가 도와드릴 수 있는 내용이 있으면 말씀해주세요.
USER > 옷 추천해줘
{
    "role": "assistant",
    "content": null,
    "function_call": {
        "name": "get_my_info",
        "arguments": "{}"
    }
}
{
    "role": "function",
    "name": "get_my_info",
    "content": "{\"sex\": \"male\", \"age\": \"44\"}"
}
{
    "role": "assistant",
    "content": null,
    "function_call": {
        "name": "get_current_weather",
        "arguments": "{\n  \"latitude\": \"37.5665\",\n  \"longitude\": \"126.9780\"\n}"
    }
}
{
    "role": "function",
    "name": "get_current_weather",
    "content": "{\"latitude\": 37.55, \"longitude\": 127.0, \"generationtime_ms\": 0.07998943328857422, \"utc_offset_seconds\": 32400, \"ti